<a href="https://colab.research.google.com/github/Amna-J/Healthcare_chatbot/blob/main/Healthcare_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv

In [13]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [14]:
from pyngrok import ngrok


ngrok.set_auth_token("TOKEN")



In [ ]:
from flask import Flask, request, jsonify, render_template_string
from openai import OpenAI


In [ ]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

SYSTEM_PROMPT = """
You are a friendly medical assistant. You ONLY respond to questions related to general health and medical topics.

Rules:
1. Do NOT give:
   - medical diagnoses
   - medicine names or doses
   - instructions for taking medicine
   - emergency instructions

2. If a question is serious or requires a doctor, respond:
   "I am not able to give medical advice. Please consult a doctor."

3. If a question is NOT medical in nature, respond firmly:
   "I can only answer medical or health-related questions. Please ask a medical question."

Always stay friendly and professional. Do not attempt to answer non-medical questions in any form.
"""

def ask_llm(question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
        ]
    )

    return response.choices[0].message.content


In [ ]:
app = Flask(__name__)

HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Healthcare Chatbot</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        /* Reset */
        * { margin: 0; padding: 0; box-sizing: border-box; }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #f5f7fa, #c3cfe2);
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
        }

        .chat-container {
            width: 100%;
            max-width: 420px;
            height: 90vh;
            background: #fff;
            border-radius: 20px;
            box-shadow: 0 15px 30px rgba(0,0,0,0.2);
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }

        .chat-header {
            background: #4f8ef7;
            color: white;
            padding: 20px;
            text-align: center;
            font-size: 1.3em;
            font-weight: bold;
            flex-shrink: 0;
        }

        #chatbox {
            flex: 1;
            padding: 15px;
            overflow-y: auto;
            display: flex;
            flex-direction: column;
            gap: 12px;
        }

        .message {
            max-width: 75%;
            padding: 12px 18px;
            border-radius: 25px;
            word-wrap: break-word;
            line-height: 1.4;
            font-size: 0.95em;
            animation: fadeIn 0.2s ease-in;
        }

        .user {
            align-self: flex-end;
            background: #4f8ef7;
            color: white;
            border-bottom-right-radius: 0;
        }

        .bot {
            align-self: flex-start;
            background: #e5e5ea;
            color: #333;
            border-bottom-left-radius: 0;
        }

        .chat-input {
            display: flex;
            border-top: 1px solid #ddd;
            flex-shrink: 0;
        }

        .chat-input input {
            flex: 1;
            padding: 16px;
            border: none;
            outline: none;
            font-size: 1em;
            border-radius: 0 0 0 20px;
        }

        .chat-input button {
            background: #4f8ef7;
            border: none;
            color: white;
            padding: 0 25px;
            font-size: 1em;
            cursor: pointer;
            border-radius: 0 0 20px 0;
            transition: background 0.2s ease;
        }

        .chat-input button:hover {
            background: #3b6ed8;
        }

        @media (max-width: 480px) {
            .chat-container {
                height: 100vh;
                border-radius: 0;
            }

            .chat-input input, .chat-input button {
                padding: 14px;
                font-size: 0.95em;
            }
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(5px); }
            to { opacity: 1; transform: translateY(0); }
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="chat-header">Healthcare Chatbot</div>
        <div id="chatbox"></div>
        <div class="chat-input">
            <input type="text" id="userInput" placeholder="Type your message...">
            <button onclick="sendMessage()">Send</button>
        </div>
    </div>
    <script>
        const chatbox = document.getElementById("chatbox");

        async function sendMessage() {
            const input = document.getElementById("userInput");
            const message = input.value.trim();
            if (!message) return;

            addMessage(message, "user");
            input.value = "";
            chatbox.scrollTop = chatbox.scrollHeight;

            try {
                const response = await fetch("/chat", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ message })
                });
                const data = await response.json();
                addMessage(data.reply, "bot");
            } catch (err) {
                addMessage("Oops! Something went wrong.", "bot");
            }

            chatbox.scrollTop = chatbox.scrollHeight;
        }

        function addMessage(text, sender) {
            const bubble = document.createElement("div");
            bubble.classList.add("message", sender);
            bubble.innerHTML = text;
            chatbox.appendChild(bubble);
        }

        document.getElementById("userInput").addEventListener("keypress", function(e) {
            if (e.key === "Enter") sendMessage();
        });
    </script>
</body>
</html>
"""


In [ ]:

for endpoint in ["home_page", "chat_page"]:
    if endpoint in app.view_functions:
        del app.view_functions[endpoint]

@app.route("/")
def home_page():
    return render_template_string(HTML)

@app.route("/chat", methods=["POST"])
def chat_page():
    data = request.json
    user_message = data.get("message", "").strip()
    if not user_message:
        return jsonify({"reply": "Please type a message."})

    reply = ask_llm(user_message)
    return jsonify({"reply": reply})


In [ ]:
from pyngrok import ngrok
import time

print("Attempting to kill all active ngrok tunnels...")
ngrok.kill()
time.sleep(3) # Increased delay to ensure cleanup
print("Ngrok tunnels killed (if any were active).")

Attempting to kill all active ngrok tunnels...
Ngrok tunnels killed (if any were active).


In [ ]:
import subprocess

print("Checking for lingering ngrok processes...")
process = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
ngrok_processes = [line for line in process.stdout.splitlines() if 'ngrok' in line and 'grep' not in line]

if ngrok_processes:
    print("Found ngrok processes:")
    for p in ngrok_processes:
        print(p)
    print("Please consider restarting the Colab runtime to fully clear these processes if they are causing issues.")
else:
    print("No active ngrok processes found.")

Checking for lingering ngrok processes...
Found ngrok processes:
root       16716  2.1  0.2 1252128 27984 ?       Sl   20:02   0:00 /root/.config/ngrok/ngrok start --none --log stdout
Please consider restarting the Colab runtime to fully clear these processes if they are causing issues.


In [ ]:
from pyngrok import ngrok
import time

ngrok.kill() # Ensure all tunnels are killed
time.sleep(1) # Add a small delay for cleanup

public_url = ngrok.connect(5002)
print("Public URL:", public_url)

app.run(port=5002)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://anarthrously-unsated-claud.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}
